<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


## Automatic Face Image Annotation & Masked Photo Generation ##

In the previous notebook, we prepared the unmasked photos in the dataset. In this notebook we will look at pre-processing the images for our masked photo dataset.

Firstly we will identify the bounding boxes for the faces as with the unmasked photos but additionally we will then also place a series of masked into those faces thus making them masked photos.

### Import required libraries

In [1]:
#!pip install lxml
#!pip install opencv-python

!wget {CONSTANTS} -P {PRE_PROCESS_DIR}
!wget {ANNOTATE} -P {PRE_PROCESS_DIR}
!wget {PLACE_MASKS} -P {PRE_PROCESS_DIR}

!pip install wget

#!pip install cmake
#!pip install dlib
!pip install face_recognition

from annotate import *
from constants import * 

from place_masks import * 


import os
import glob
import shutil

import cv2
import numpy as np
from lxml import etree as ET
from PIL import Image, ImageFile

import sys
import random
import face_recognition



--2020-06-10 17:46:16--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3684 (3.6K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   3.60K  --.-KB/s    in 0s      

2020-06-10 17:46:17 (53.9 MB/s) - ‘constants.py’ saved [3684/3684]

--2020-06-10 17:46:18--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/annotate.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2658 (2

### Prepare dataset ###

#### Connect to Google Drive


In [0]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Download faces dataset and extract to folder

In [2]:
os.chdir(DATASET_DIR_TMP1)
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/1k_faces_01.zip
!unzip -j 1k_faces_01.zip '*.jpg'
!rm 1k_faces_01.zip


--2020-06-10 17:50:40--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/1k_faces_01.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23768454 (23M) [application/zip]
Saving to: ‘1k_faces_01.zip’

1k_faces_01.zip     100%[===================>]  22.67M  68.4MB/s    in 0.3s    

2020-06-10 17:50:42 (68.4 MB/s) - ‘1k_faces_01.zip’ saved [23768454/23768454]

Archive:  1k_faces_01.zip
  inflating: 00SD82OK2A.jpg          
  inflating: 00V5CZZSSO.jpg          
  inflating: 01P734HDO0.jpg          
  inflating: 02IWA7SQR9.jpg          
  inflating: 02PO4NZNKS.jpg          
  inflating: 02VVQ8EUPO.jpg          
  inflating: 0393CKG05Z.jpg          
  inflating: 04W6LHK65S.jpg          
  inflating: 062U7MAFP7.jpg          
  inflating

#### Annotate the images
Calls the annotate function to automatically annotate the faces in our image folder and outputs XML files

In [0]:
annotate(DATASET_DIR_TMP1,DATASET_DIR_TMP2)

## Generate Masked Photos

### Download required dependency and extract to folder

In [10]:
os.chdir(DATASET_DIR_UNPREP)

import tarfile
tf = tarfile.open(wget.download(MASKS))
tf.extractall()
#!tar -xf mask_photos.tar.xz -C {DATASET_DIR_UNPREP_MASKS}

 annotations/   images/  'mask_photos.tar (1).xz'   mask_photos.tar.xz


### Define environment

In [12]:
model = "hog" # Select facial recognition model; Available options areh 'hog' or 'cnn'. Hog is faster, cnn is more accurate.

#num_generated_files = 0
input_folder_list = [os.path.join(DATASET_DIR_TMP1, f) for f in os.listdir(DATASET_DIR_TMP1) if os.path.isfile(os.path.join(DATASET_DIR_TMP1, f))]
mask_folder_list = [os.path.join(DATASET_DIR_UNPREP_MASKS, f) for f in os.listdir(DATASET_DIR_UNPREP_MASKS) if os.path.isfile(os.path.join(DATASET_DIR_UNPREP_MASKS, f))]

for i in range(len(input_folder_list)):
     # get a random mask from from the mask folder
     random_mask_file = random.choice(list(mask_folder_list))
     random_mask_file = os.getcwd() + "/" + random_mask_file
     
     if random_mask_file.endswith(('.png', '.jpg', '.jpeg', '.xml')): 
      # get the next image from the image folder
      current_image_file = os.getcwd() + "/" + input_folder_list[i]
      create_mask(current_image_file, random_mask_file, model)



#input_folder_path = os.path.join(os.getcwd(), input_folder)
#mask_folder_path = os.path.join(os.getcwd(), mask_folder)

FileNotFoundError: ignored

### Helper code to place masks on the faces

In [0]:


def save(face_image_file, current_image_file):
    face_image_file.save(current_image_file)
    print(f'Save to {current_image_file}')
    

def create_mask(current_image_file, current_mask_file, model):
    show = False

    KEY_FACIAL_FEATURES = ('nose_bridge', 'chin')
    face_image_file: ImageFile = None
    mask_image_file: ImageFile = None

    face_image_np = face_recognition.load_image_file(current_image_file)
    face_locations = face_recognition.face_locations(
        face_image_np, model=model)
    face_landmarks = face_recognition.face_landmarks(
        face_image_np, face_locations)
    face_image_file = Image.fromarray(face_image_np)
    mask_image_file = Image.open(current_mask_file)

    found_face = False
    for face_landmark in face_landmarks:

        # check whether facial features meet requirement
        skip = False
        for facial_feature in KEY_FACIAL_FEATURES:
            if facial_feature not in face_landmark:
                skip = True
                break
            if skip:
                continue

            # mask face
            found_face = True
            place_mask(face_landmark, mask_image_file, face_image_file)

    if found_face:
        if show:
            face_image_file.show()

        # save
        save(face_image_file,current_image_file)
    else:
        print('Found no face.')


def place_mask(face_landmark: dict, mask_image_file, face_image_file):
    nose_bridge = face_landmark['nose_bridge']
    nose_point = nose_bridge[len(nose_bridge) * 1 // 4]
    nose_v = np.array(nose_point)

    chin = face_landmark['chin']
    chin_len = len(chin)
    chin_bottom_point = chin[chin_len // 2]
    chin_bottom_v = np.array(chin_bottom_point)
    chin_left_point = chin[chin_len // 8]
    chin_right_point = chin[chin_len * 7 // 8]

    # split mask and resize
    width = mask_image_file.width
    height = mask_image_file.height
    width_ratio = 1.2
    new_height = int(np.linalg.norm(nose_v - chin_bottom_v))

    # left
    mask_left_img = mask_image_file.crop((0, 0, width // 2, height))
    mask_left_width = get_distance_from_point_to_line(
        chin_left_point, nose_point, chin_bottom_point)
    mask_left_width = int(mask_left_width * width_ratio)
    mask_left_img = mask_left_img.resize((mask_left_width, new_height))

    # right
    mask_right_img = mask_image_file.crop((width // 2, 0, width, height))
    mask_right_width = get_distance_from_point_to_line(
        chin_right_point, nose_point, chin_bottom_point)
    mask_right_width = int(mask_right_width * width_ratio)
    mask_right_img = mask_right_img.resize((mask_right_width, new_height))

    # merge mask
    size = (mask_left_img.width + mask_right_img.width, new_height)
    mask_img = Image.new('RGBA', size)
    mask_img.paste(mask_left_img, (0, 0), mask_left_img)
    mask_img.paste(mask_right_img, (mask_left_img.width, 0), mask_right_img)

    # rotate mask
    angle = np.arctan2(
        chin_bottom_point[1] - nose_point[1], chin_bottom_point[0] - nose_point[0])
    rotatedmask_image_file = mask_img.rotate(angle, expand=True)

    # calculate mask location
    center_x = (nose_point[0] + chin_bottom_point[0]) // 2
    center_y = (nose_point[1] + chin_bottom_point[1]) // 2

    offset = mask_img.width // 2 - mask_left_img.width
    radian = angle * np.pi / 180
    box_x = center_x + int(offset * np.cos(radian)) - \
        rotatedmask_image_file.width // 2
    box_y = center_y + int(offset * np.sin(radian)) - \
        rotatedmask_image_file.height // 2

    # add mask
    face_image_file.paste(mask_img, (box_x, box_y), mask_img)


def get_distance_from_point_to_line(point, line_point1, line_point2):
    distance = np.abs((line_point2[1] - line_point1[1]) * point[0] +
                      (line_point1[0] - line_point2[0]) * point[1] +
                      (line_point2[0] - line_point1[0]) * line_point1[1] +
                      (line_point1[1] - line_point2[1]) * line_point1[0]) / \
        np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
    return int(distance)


#### We loop through all the files in our input directory, place masks on the faces and the save them in the output directory

In [0]:
for i in range(len(input_folder_list)):
     # get a random mask from from the mask folder
     random_mask_file = random.choice(list(mask_folder_list))
     random_mask_file = os.getcwd() + "/" + random_mask_file
     
     if random_mask_file.endswith(('.png', '.jpg', '.jpeg', '.xml')): 
      # get the next image from the image folder
      current_image_file = os.getcwd() + "/" + input_folder_list[i]
      create_mask(current_image_file, random_mask_file, model)


Save to /content/dataset/masked/WHHC578I6K.jpg
Save to /content/dataset/masked/XCYCNCMDIF.jpg
Save to /content/dataset/masked/GAW0VO7OTB.jpg
Save to /content/dataset/masked/T5AD07K975.jpg
Save to /content/dataset/masked/0PKNKA4WR6.jpg
Save to /content/dataset/masked/E4ZTFZD4MW.jpg
Save to /content/dataset/masked/9NU2BBTWZW.jpg
Save to /content/dataset/masked/PEWEFU6VFO.jpg
Save to /content/dataset/masked/Z7I561ACNM.jpg
Save to /content/dataset/masked/HLNGKGIVQ5.jpg
Save to /content/dataset/masked/YRLZFEQDG9.jpg
Save to /content/dataset/masked/3QXE62VWSO.jpg
Save to /content/dataset/masked/X76STDZ3DI.jpg
Save to /content/dataset/masked/PTE1ZN0MZ2.jpg
Save to /content/dataset/masked/9S2JTR4S1S.jpg
Save to /content/dataset/masked/5OYYEYFED0.jpg
Save to /content/dataset/masked/L41K4XO4F3.jpg
Save to /content/dataset/masked/9XGBZIBQZE.jpg
Save to /content/dataset/masked/KFJO394SLP.jpg
Save to /content/dataset/masked/32SS01SQ0U.jpg
Save to /content/dataset/masked/KF8YUJHEI9.jpg
Save to /cont

#### Save the generated images and archive them

In [0]:
def move_files(source,destination):
    files = os.listdir(source)
    for f in files:
        if f.endswith(('.png', '.jpg', '.jpeg', '.xml')):
          shutil.move(source+"/"+f, destination+"/")

# Move the images and the annotations from the temp 
move_files(DATASET_DIR_TMP1,DATASET_DIR_UNPREP_IMG)
move_files(DATASET_DIR_TMP2,DATASET_DIR_UNPREP_ANNO)

In [0]:
dataset_labels_folder = input_folder_path + "-labels"
source1 = input_folder_path + "/"
source2 = dataset_labels_folder  + "/"

destination = os.getcwd() + "/training_demo"+ "/images"

if not os.path.exists(destination):
    os.makedirs(destination)


def move_files(source,destination):
    files = os.listdir(source)
    for f in files:
        shutil.move(source+f, destination)


move_files(source1,destination)
move_files(source2,destination)




### Compress Dataset and Save Dataset to Google Drive

Follow this step if you wish to archive your dataset for future use

In [0]:
!zip -r masked-datasetv2.zip {DATASET_DIR_UNPREP}
!gsutil cp masked-datasetv2.zip {DRIVE_DEV}

  adding: content/training_demo/images/ (stored 0%)
  adding: content/training_demo/images/EIZSKIRIT6.xml (deflated 47%)
  adding: content/training_demo/images/WHHC578I6K.jpg (deflated 1%)
  adding: content/training_demo/images/M5XU02T5UA.xml (deflated 47%)
  adding: content/training_demo/images/XJ1BCWDGBX.xml (deflated 47%)
  adding: content/training_demo/images/XCYCNCMDIF.jpg (deflated 1%)
  adding: content/training_demo/images/UBLHJMVTA7.xml (deflated 47%)
  adding: content/training_demo/images/GAW0VO7OTB.jpg (deflated 1%)
  adding: content/training_demo/images/T5AD07K975.jpg (deflated 1%)
  adding: content/training_demo/images/0PKNKA4WR6.jpg (deflated 1%)
  adding: content/training_demo/images/E4ZTFZD4MW.jpg (deflated 1%)
  adding: content/training_demo/images/8N6KGKVX2L.xml (deflated 47%)
  adding: content/training_demo/images/9NU2BBTWZW.jpg (deflated 1%)
  adding: content/training_demo/images/VQ0PNYDJV9.xml (deflated 46%)
  adding: content/training_demo/images/KLQ35JH2NE.xml (def